In [1]:
!pip install tensorflow==2.3.0

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import expand_dims

img = load_img("Shape Dataset/Train/Circles/Circle0.jpg")
wordLabelCircle = img.split('.')
print(wordLabelCircle)

In [4]:
augmentation=0
while augmentation < 3:
    
    circleTrainDatagen = ImageDataGenerator(rotation_range=15,zoom_range=0.1,rescale=1./255)
    CircleTrainPath="Shape Dataset/Train/Circle/"
    circleTrain = os.listdir(CircleTrainPath)

    for index, image in enumerate(circleTrain):          # for loop for training images of Circle. 
        filename = image.split('.')
        if(filename[1] == 'png' or filename[1] == 'jpg' or filename[1] == 'jpeg'):
            # load the image using split concept.
            img = load_img(CircleTrainPath+filename[0]+'.'+filename[1])     
            # convert to numpy array
            data = img_to_array(img)
            # expand dimension to one sample
            circleTrainSamples = expand_dims(data, axis=0)

            circleTrainIT = circleTrainDatagen.flow(circleTrainSamples, batch_size=32,save_to_dir= CircleTrainPath,
                              save_prefix="CIRCLE",
                              save_format='jpg')     #Using datagen.flow, every image is flipped horizontally, 
                                                    #rotated at certain angle and finally stored in a specified directory. 
            batch = circleTrainIT.next()

        #Same concept follows for test images of Square shown below

    squareTrainDatagen = ImageDataGenerator(zoom_range=0.1,rescale=1./255)
    SquareTrainPath="Shape Dataset/Train/Square/"
    squareTrain = os.listdir(SquareTrainPath)

    for index, image in enumerate(squareTrain):           
        filename = image.split('.')
        if(filename[1] == 'png' or filename[1] == 'jpg' or filename[1] == 'jpeg'):
            img = load_img(SquareTrainPath+filename[0]+'.'+filename[1])
            data = img_to_array(img)
            squareTrainSamples = expand_dims(data, axis=0)
            squareTrainIt = squareTrainDatagen.flow(squareTrainSamples, batch_size=32,save_to_dir= SquareTrainPath,
                              save_prefix="SQUARE",
                              save_format='jpg')     #Using datagen.flow, every image is flipped horizontally, 
                                                    #rotated at certain angle and finally stored in a specified directory. 
            batch = squareTrainIt.next()
    
    augmentation+=1

In [5]:
augmentation=0
while augmentation < 3:
    
    circleValidationDatagen = ImageDataGenerator(rotation_range=15,zoom_range=0.1,rescale=1./255)
    CircleValidationPath="Shape Dataset/Validation/Circle/"
    circleValidation = os.listdir(CircleTrainPath)

    for index, image in enumerate(circleTrain):          # for loop for training images of Circle. 
        filename = image.split('.')
        if(filename[1] == 'png' or filename[1] == 'jpg' or filename[1] == 'jpeg'):
            # load the image using split concept.
            img = load_img(CircleTrainPath+filename[0]+'.'+filename[1])
            # convert to numpy array
            data = img_to_array(img)
            # expand dimension to one sample
            circleValidationSamples = expand_dims(data, axis=0)

            circleValidationIT = circleValidationDatagen.flow(circleValidationSamples, batch_size=32,save_to_dir= CircleValidationPath,
                              save_prefix="CIRCLE",
                              save_format='jpg')     #Using datagen.flow, every image is flipped horizontally, 
                                                    #rotated at certain angle and finally stored in a specified directory. 
            batch = circleValidationIT.next()

        #Same concept follows for test images of Square shown below

    squareValidationDatagen = ImageDataGenerator(zoom_range=0.1,rescale=1./255)
    SquareValidationPath="Shape Dataset/Validation/Square/"
    squareValidation = os.listdir(SquareTrainPath)

    for index, image in enumerate(squareTrain):           
        filename = image.split('.')
        if(filename[1] == 'png' or filename[1] == 'jpg' or filename[1] == 'jpeg'):
            img = load_img(SquareTrainPath+filename[0]+'.'+filename[1])
            data = img_to_array(img)
            squareValidationSamples = expand_dims(data, axis=0)


            squareValidationIt = squareValidationDatagen.flow(squareValidationSamples, batch_size=32,save_to_dir= SquareValidationPath,
                              save_prefix="SQUARE",
                              save_format='jpg')     #Using datagen.flow, every image is flipped horizontally, 
                                                    #rotated at certain angle and finally stored in a specified directory. 
            batch = squareValidationIt.next()
    
    augmentation+=1

In [6]:
trainDatagen = ImageDataGenerator(rescale=1./255)
 
testDatagen = ImageDataGenerator(rescale=1./255)
 
train_generator = trainDatagen.flow_from_directory(
    "Shape Dataset/Train",
    target_size=(224,224),
    batch_size=32,
    class_mode='binary')
 
validation_generator = testDatagen.flow_from_directory(
    "Shape Dataset/Validation",
    target_size=(224,224),
    batch_size=32,
    class_mode='binary')

Found 595 images belonging to 2 classes.
Found 953 images belonging to 2 classes.


In [12]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD

#VGG16 Architecture used for Convolution of images

base_model=tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None
)

for layer in base_model.layers:
        layer.trainable = False

x=base_model.output

model=Flatten()(x)
model=Dense(units=5096,activation="relu")(model)
model=Dropout(0.5)(model)
model=Dense(units=5096,activation="relu")(model)
output=Dense(units=1, activation="sigmoid")(model)

model = keras.models.Model(inputs=base_model.input, outputs = output)

from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [13]:
model.fit_generator(steps_per_epoch=1,generator=train_generator,validation_data=validation_generator,validation_steps=1,epochs=50,verbose=1,shuffle=True)

Epoch 1/50
1/1 [==============================] - 6s 6s/step - loss: 0.6930 - accuracy: 0.6562 - val_loss: 0.6705 - val_accuracy: 0.6250
Epoch 2/50
1/1 [==============================] - 5s 5s/step - loss: 0.6947 - accuracy: 0.5312 - val_loss: 0.6784 - val_accuracy: 0.5625
Epoch 3/50
1/1 [==============================] - 5s 5s/step - loss: 0.6773 - accuracy: 0.5938 - val_loss: 0.7224 - val_accuracy: 0.4688
Epoch 4/50
1/1 [==============================] - 5s 5s/step - loss: 0.7685 - accuracy: 0.3438 - val_loss: 0.6725 - val_accuracy: 0.6562
Epoch 5/50
1/1 [==============================] - 5s 5s/step - loss: 0.7937 - accuracy: 0.4688 - val_loss: 0.6522 - val_accuracy: 0.6250
Epoch 6/50
1/1 [==============================] - 6s 6s/step - loss: 0.7322 - accuracy: 0.5000 - val_loss: 0.7152 - val_accuracy: 0.4375
Epoch 7/50
1/1 [==============================] - 6s 6s/step - loss: 0.7580 - accuracy: 0.3438 - val_loss: 0.6743 - val_accuracy: 0.6250
Epoch 8/50
1/1 [=========================

In [14]:
_, trainacc = model.evaluate(train_generator, verbose=1)
print("\n\t Accuracy")
print('> %.3f' % (trainacc * 100.0))

19/19 [==============================] - 107s 6s/step - loss: 0.6602 - accuracy: 0.6134

	 Accuracy
> 61.345


In [15]:
_, validationacc = model.evaluate(validation_generator, verbose=1)
print("\n\t Accuracy")
print('> %.3f' % (validationacc * 100.0))     

30/30 [==============================] - 175s 6s/step - loss: 0.6618 - accuracy: 0.6128

	 Accuracy
> 61.280


In [18]:
model.save("shapes_model.h5")

Reference:

https://stackoverflow.com/questions/62409838/error-in-loading-image-dataset-from-directory-in-tensorflow
